In [58]:
# 빅콘

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib
matplotlib.rcParams['font.family'].insert(0, 'Malgun Gothic') 

In [2]:
# 데이터 불러오기
실적데이터1 = pd.read_excel('../01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx', header=1)
평가데이터1 = pd.read_excel('../02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', header=1)
시청률데이터1 = pd.read_excel('../01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx', header=1)

# 외부데이터 강수량, 풍속, 기온 데이터
강수량1 = pd.read_csv('../01_제공데이터/강수량.csv', encoding='euc-kr')
풍속1 = pd.read_csv('../01_제공데이터/풍속.csv', encoding='euc-kr')
기온1 = pd.read_csv('../01_제공데이터/기온.csv', encoding='euc-kr')
# 시청률데이터1 전처리

# 행과 열의 마지막 잘라내기
시청률데이터1 = 시청률데이터1.iloc[:-1, :-1]

# 시간대 뒤에 ':00붙이기'
시청률데이터1['시간대'] = 시청률데이터1['시간대'] + ':00'

# 시간대열 인덱스로 바꾸기
시청률데이터1.set_index('시간대', inplace=True)

In [3]:
# ----------------------------------------------------------------------------------------------------------------
강수량 = 강수량1.sort_values(by = '일시')
풍속 = 풍속1.sort_values(by = '일시')
기온 = 기온1.sort_values(by = '일시')

풍속.일시=pd.to_datetime(풍속.일시).map(lambda x : str(x).split(" ")[0])

강수량 = 강수량[['일시', '평균일강수량(mm)']]
풍속 = 풍속[['일시', '최대풍속(m/s)']]
기온 = 기온[['일시', '평균최고기온(℃)', '평균최저기온(℃)']]

강수량.index = list(range(365))
풍속.index = list(range(365))
기온.index = list(range(365))

# 여기까지 결측치 처리 ( 노출 = 윗 값으로, 취급액 = 0으로)
# 이후로 전처리

In [4]:
실적데이터 = 실적데이터1.copy()
시청률데이터 = 시청률데이터1.copy()
평가데이터 = 평가데이터1.copy()

In [5]:
실적데이터

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


In [6]:
# 컬럼명 변경
실적데이터.columns = ['방송일시', '노출', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액']

# 결측치 제거
실적데이터.취급액 = 실적데이터.취급액.fillna(0)
실적데이터.노출 = 실적데이터.노출.fillna(method = 'ffill')

# 2020 데이터 지우기
실적데이터 = 실적데이터[:38300]

# 방송일시로부터 시간 데이터 열로 추가
dt = datetime.datetime.now()

실적데이터['요일'] = 실적데이터['방송일시'].dt.dayofweek
실적데이터['월']  = 실적데이터['방송일시'].dt.month
실적데이터['주'] = 실적데이터['방송일시'].dt.week
실적데이터['시간']  = 실적데이터['방송일시'].dt.time
실적데이터['시']  = 실적데이터['방송일시'].dt.hour
실적데이터['일'] = 실적데이터['방송일시'].dt.day

실적데이터['방송시간'] = 실적데이터.방송일시.apply(lambda x : str(x)).apply(lambda x : x.split(' ')[1][:])
실적데이터['시간'] = 실적데이터.시간.apply(lambda x : str(x)).apply(lambda x : int(x.split(':')[0])*60 + int(x.split(':')[1]))
실적데이터['일시'] = 실적데이터.방송일시.apply(lambda x : str(x)).apply(lambda x : x.split(' ')[0])

D:\ringa\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\ringa\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
D:\ringa\anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  app.launch_new_instance()
D:\ringa\anaconda3\lib\site-packages

In [7]:
분당평균시청률=[]
for i in range(len(실적데이터)):
    시작=실적데이터.방송시간.iloc[i]
    종료=str(pd.to_datetime(실적데이터.방송시간.iloc[i])+datetime.timedelta(minutes=round(float(실적데이터.노출.iloc[i]),0)-1)).split(" ")[1][:]
    분당평균시청률.append((시청률데이터[실적데이터.일시.iloc[i]].loc[시작:종료].mean()))
    

실적데이터['분당평균시청률']=분당평균시청률

D:\ringa\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# 실적데이터에 강수량 평균, 기온 평균 최고 기온/ 평균 최저 기온, 최대 풍속 추가
실적데이터 = pd.merge(실적데이터, 강수량, on='일시' )
실적데이터 = pd.merge(실적데이터, 기온, on='일시')
실적데이터 = pd.merge(실적데이터, 풍속, on='일시')

# 당장 못쓰는 방송시간 관련 컬럼 삭제
실적데이터.drop(['방송시간', '방송일시', '일시'], axis=1, inplace=True)

# 분당평균시청률 더 잘 보기(* 100)
실적데이터['분당평균시청률'] = round(실적데이터['분당평균시청률'] * 100, 3)

# 상품명 컬럼 삭제
실적데이터.drop('상품명', axis=1, inplace=True)

# 주문량
실적데이터['주문량'] = (실적데이터.취급액 / 실적데이터.판매단가).fillna(0)

In [9]:
실적데이터

,노출,마더코드,상품코드,상품군,판매단가,취급액,요일,월,주,시간,시,일,분당평균시청률,평균일강수량(mm),평균최고기온(℃),평균최저기온(℃),최대풍속(m/s),주문량
0,20.0,100346,201072,의류,39900,2099000.0,1,1,1,360,6,1,0.00,0.1,2.1,-5.8,15.0,52.606516
1,20.0,100346,201079,의류,39900,4371000.0,1,1,1,360,6,1,0.00,0.1,2.1,-5.8,15.0,109.548872
2,20.0,100346,201072,의류,39900,3262000.0,1,1,1,380,6,1,0.00,0.1,2.1,-5.8,15.0,81.754386
3,20.0,100346,201079,의류,39900,6955000.0,1,1,1,380,6,1,0.00,0.1,2.1,-5.8,15.0,174.310777
4,20.0,100346,201072,의류,39900,6672000.0,1,1,1,400,6,1,0.00,0.1,2.1,-5.8,15.0,167.218045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38295,20.0,100448,201391,주방,148000,1664000.0,1,12,1,1400,23,31,0.00,0.1,0.4,-6.7,21.9,11.243243
38296,20.0,100448,201383,주방,178000,9149000.0,1,12,1,1420,23,31,2.06,0.1,0.4,-6.7,21.9,51.398876
38297,20.0,100448,201390,주방,168000,15282000.0,1,12,1,1420,23,31,2.06,0.1,0.4,-6.7,21.9,90.964286
38298,20.0,100448,201384,주방,158000,2328000.0,1,12,1,1420,23,31,2.06,0.1,0.4,-6.7,21.9,14.734177


In [10]:
# ---------------------------------------------- 파생변수 ------------------------------------------------------------
# 수목금토 / 일월화 칼럼 추가
실적데이터['수목금토'] = 실적데이터.요일.map(lambda X: X in [3,4,5,6]).map(lambda X: 1 if X else 0)

# 계절 (봄 345 여름 678 가을 91011 겨울 1212) 의미 크게 없음 / 상,하반기
실적데이터['월789'] = 실적데이터.월.map(lambda X: X in [7, 8, 9]).map(lambda X: 1 if X else 0)
#실적데이터['하반기'] = 실적데이터.월.map(lambda X: X in [7,8,9,10,11,12]).map(lambda X: 1 if X else 0)
# 실적데이터['가을'] = 실적데이터.월.map(lambda X: X in [9,10,11]).map(lambda X: 1 if X else 0)
# 실적데이터['겨울'] = 실적데이터.월.map(lambda X: X in [12,1,2]).map(lambda X: 1 if X else 0)

# 일 : 월초,말이랑 중간 비교 / 의미없음
#실적데이터['월말초'] = 실적데이터.일.map(lambda X: X in [1,2,3,4,5,6,27,28,29,30,31]).map(lambda X: 1 if X else 0)

# 판매단가
실적데이터['저가'] = 실적데이터.판매단가.map(lambda X: 1 if X <=30000 else 0)
실적데이터['중가'] = 실적데이터.판매단가.map(lambda X: 1 if 30000< X <= 400000  else 0)
실적데이터['고가'] = 실적데이터.판매단가.map(lambda X: 1 if X >400000  else 0)

# 노출 시간 
실적데이터['노출20미만'] = 실적데이터.노출.map(lambda X: 1 if X < 20  else 0)
실적데이터['노출2030'] = 실적데이터.노출.map(lambda X: 1 if 20 <= X < 30  else 0)
실적데이터['노출30이상'] = 실적데이터.노출.map(lambda X: 1 if X >= 30  else 0)

# 주 32주 전후
실적데이터['주32후'] = 실적데이터.주.map(lambda X: 1 if X >= 32  else 0)

# 시간별 상품코드 배치에 따른 시간 분리 
실적데이터['시간1015사이20이후'] = 실적데이터.시.map(lambda X: 1 if X >= 20 or 10 < X < 15  else 0)
#실적데이터['시간6-9'] = 실적데이터.노출.map(lambda X: 1 if 6 < X < 21  else 0)

In [11]:
list(실적데이터.columns)

['노출',
 '마더코드',
 '상품코드',
 '상품군',
 '판매단가',
 '취급액',
 '요일',
 '월',
 '주',
 '시간',
 '시',
 '일',
 '분당평균시청률',
 '평균일강수량(mm)',
 '평균최고기온(℃)',
 '평균최저기온(℃)',
 '최대풍속(m/s)',
 '주문량',
 '수목금토',
 '월789',
 '저가',
 '중가',
 '고가',
 '노출20미만',
 '노출2030',
 '노출30이상',
 '주32후',
 '시간1015사이20이후']

In [12]:
#-------------------------------------------통째로 칼럼화---------------------------------------------------------
# 요일 칼럼화
# 요일_0 ~ 요일_6
실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.요일, prefix='요일')], axis=1)
실적데이터.drop('요일', axis=1, inplace=True)

# 월 칼럼화
# 1 ~ 12 -> 월_1, 월_2, ~ ,월_12
실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.월, prefix='월')], axis=1)
실적데이터.drop('월', axis=1, inplace=True)

# 일 칼럼화
# 1 ~ 31 -> 일_1, 일_2, ~, 일_31
실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.일, prefix='일')], axis=1)
실적데이터.drop('일', axis=1, inplace=True)

# 시 칼럼화
# 1 ~ 23 -> 시_1, 시_2, ~, 시_23
실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.시, prefix='시')], axis=1)
실적데이터.drop('시', axis=1, inplace=True)

# 주 칼럼화
# 1 ~ 52 -> 주_1, 주_2, ~, 주_52
실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.주, prefix='주')], axis=1)
실적데이터.drop('주', axis=1, inplace=True)

# 상품군 컬럼화
# ['가구', '가전', '건강기능', '농수축', '무형', '생활용품', '속옷', '의류', '이미용', '잡화','주방', '침구']
실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.상품군)], axis=1)
실적데이터.drop('상품군', axis=1, inplace=True)

# 상품코드 칼럼화
# 1 ~ 52 -> 상품코드_1, 상품코드_2, ~, 상품코드_??
#실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.상품코드, prefix='상품코드')], axis=1)
#실적데이터.drop('상품코드', axis=1, inplace=True)

# Data Split

In [147]:
# 상품군 컬럼화
# ['가구', '가전', '건강기능', '농수축', '무형', '생활용품', '속옷', '의류', '이미용', '잡화','주방', '침구']
실적데이터 = pd.concat([실적데이터, pd.get_dummies(실적데이터.상품군)], axis=1)
실적데이터.drop('상품군', axis=1, inplace=True)

AttributeError: 'DataFrame' object has no attribute '상품군'

In [13]:
# 독립변수 = 취급액, 분당평균시청률, 주문량 제외하고 전부
# 종속변수 = 주문량
실적데이터_test = 실적데이터
X_features = 실적데이터_test.drop(['취급액','분당평균시청률','주문량', '마더코드','시간'], axis=1)
y_target = 실적데이터_test.주문량

# train test split
X_train, X_test, y_train, y_test = \
train_test_split(X_features, y_target ,test_size=0.3, random_state=0)

In [399]:
# z정규화(표준편차를 이용해서 이상치의 영향을 덜 받는 정규화)

def z_score_normalize(lst):
    normalized = []
    for value in lst:
        normalized_num = (value - np.mean(lst)) / np.std(lst)
        normalized.append(normalized_num)
    return normalized

# z 정규화 적용후 다시 스플릿

import numpy as np
# 기존 데이터프레임 전처리
X_features = 실적데이터.drop('취급액', axis=1)
for i in X_features.columns:
    X_features[i] = z_score_normalize(X_features[i])
    
X_train, X_test, y_train, y_test = \
train_test_split(X_features, y_target ,test_size=0.3, random_state=0)

KeyboardInterrupt: 

In [460]:
# 0~1 정규화
from sklearn.preprocessing import MinMaxScaler

minmaxScaler = MinMaxScaler()
X_features = minmaxScaler.fit_transform(X_features)
y_target = minmaxScaler.fit_transform(np.array(y_target).reshape(-1,1))
X_features

array([[0.40753828, 0.42658177, 0.63636364, ..., 0.25174825, 0.26086957,
        0.        ],
       [0.40753828, 0.42936729, 0.63636364, ..., 0.25174825, 0.26086957,
        0.        ],
       [0.40753828, 0.42658177, 0.63636364, ..., 0.26573427, 0.26086957,
        0.        ],
       ...,
       [0.52767962, 0.55312376, 0.90909091, ..., 0.99300699, 1.        ,
        0.16401274],
       [0.52767962, 0.55073617, 0.90909091, ..., 0.99300699, 1.        ,
        0.16401274],
       [0.52767962, 0.55352169, 0.90909091, ..., 0.99300699, 1.        ,
        0.16401274]])

# XGBoost 

In [14]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators=500, gamma=0, learning_rate=0.4, max_depth=10, n_jobs=-1, random_state=1)

xgb_reg.fit(X_train, y_train) # early_stopping_rounds=200,
         # eval_set = [(X_test, y_test)], eval_metric='logloss')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.4, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=-1, num_parallel_tree=1, random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
xgb_reg.score(X_test, y_test)

0.6736563785705119

In [16]:
X_train

,노출,상품코드,판매단가,평균일강수량(mm),평균최고기온(℃),평균최저기온(℃),최대풍속(m/s),수목금토,월789,저가,...,건강기능,농수축,무형,생활용품,속옷,의류,이미용,잡화,주방,침구
21387,20.000000,200435,1330000,56.5,27.1,23.2,16.1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
18836,20.000000,202444,59900,12.0,28.8,20.6,9.2,1,0,0,...,0,1,0,0,0,0,0,0,0,0
35534,20.000000,202430,0,0.1,9.5,-0.3,11.5,0,0,1,...,0,0,1,0,0,0,0,0,0,0
7001,16.200000,200246,59000,0.0,9.4,-1.5,16.4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
26422,14.833333,200944,178000,12.3,29.1,22.6,42.1,1,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20757,20.000000,202474,268000,8.8,27.1,20.6,6.0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
32103,20.000000,201173,198000,0.0,18.3,5.7,10.1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
30403,20.000000,201177,148000,0.0,22.3,10.6,12.1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
21243,20.000000,200583,54900,13.9,28.5,22.1,14.8,1,1,0,...,0,1,0,0,0,0,0,0,0,0


In [17]:
y_train

21387      0.000000
18836    431.268781
35534      0.000000
7001     156.423729
26422     44.174157
            ...    
20757     41.082090
32103    108.479798
30403     39.229730
21243    901.347905
2732     101.511628
Name: 주문량, Length: 26810, dtype: float64

In [21]:
실적데이터_test.to_csv('실적데이터_test.csv', encoding='euc-kr')

# AdaBoost

In [104]:
from sklearn.ensemble import AdaBoostRegressor
import  sklearn.metrics as mt

ada_reg = AdaBoostRegressor(base_estimator=None, learning_rate=0.05, loss='exponential',
         n_estimators=100, random_state=0) 

ada_reg.fit(X_train, y_train)

y_pred = ada_reg.predict(X_test)

In [105]:
mt.r2_score(y_test, y_pred)

0.4811364334083612

# KNN

In [106]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor(n_neighbors = 12, weights = "distance")
knn_reg.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=12, weights='distance')

In [107]:
y_pred = knn_reg.predict(X_test)

In [108]:
mt.r2_score(y_test, y_pred)

0.6748515234323371

# LGBMRegressor

In [109]:
# conda install lightgbm

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [110]:
lgbm_reg = LGBMRegressor()

In [111]:
len(X_train.columns)

152

In [112]:
cols = []
for i in range(152):
        cols.append(str(i))
X_train.columns = cols

In [113]:
lgbm_reg.fit(X_train, y_train)

LGBMRegressor()

In [114]:
y_pred = lgbm_reg.predict(X_test)

In [115]:
mt.r2_score(y_test, y_pred)

0.733002194911559

In [193]:
X_train.columns = ['마더코드', '상품코드', '판매단가', '시간', '분당평균시청률', '월요일', '화요일', '수요일', '목요일',
       '금요일', '토요일', '일요일', '월_1', '월_2', '월_3', '월_4', '월_5', '월_6', '월_7',
       '월_8', '월_9', '월_10', '월_11', '월_12', '일_1', '일_2', '일_3', '일_4', '일_5',
       '일_6', '일_7', '일_8', '일_9', '일_10', '일_11', '일_12', '일_13', '일_14',
       '일_15', '일_16', '일_17', '일_18', '일_19', '일_20', '일_21', '일_22', '일_23',
       '일_24', '일_25', '일_26', '일_27', '일_28', '일_29', '일_30', '일_31', '시_0',
       '시_1', '시_2', '시_6', '시_7', '시_8', '시_9', '시_10', '시_11', '시_12',
       '시_13', '시_14', '시_15', '시_16', '시_17', '시_18', '시_19', '시_20', '시_21',
       '시_22', '시_23', '가구', '가전', '건강기능', '농수축', '무형', '생활용품', '속옷', '의류',
       '이미용', '잡화', '주방', '침구']

# 심층신경망

In [304]:
X_train

,노출,판매단가,수목금토,월789,저가,중가,고가,노출20미만,노출2030,노출30이상,...,상품코드_202501,상품코드_202502,상품코드_202503,상품코드_202504,상품코드_202505,상품코드_202508,상품코드_202509,상품코드_202510,상품코드_202512,상품코드_202513
21387,20.000000,1330000,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
18836,20.000000,59900,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
35534,20.000000,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7001,16.200000,59000,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26422,14.833333,178000,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20757,20.000000,268000,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
32103,20.000000,198000,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
30403,20.000000,148000,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21243,20.000000,54900,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 0~1 정규화
from sklearn.preprocessing import MinMaxScaler

minmaxScaler = MinMaxScaler()
X_features = minmaxScaler.fit_transform(X_features)
y_target = minmaxScaler.fit_transform(np.array(y_target).reshape(-1,1))
X_features

In [272]:
X_train

,노출,상품코드,판매단가,요일,월,시간,시,일,수목금토,상반기,...,건강기능,농수축,무형,생활용품,속옷,의류,이미용,잡화,주방,침구
21387,20.000000,200435,1330000,5,7,1280,21,20,1,0,...,0,0,0,0,0,0,0,0,0,0
18836,20.000000,202444,59900,3,6,1080,18,27,1,1,...,0,1,0,0,0,0,0,0,0,0
35534,20.000000,202430,0,2,12,1080,18,4,0,0,...,0,0,1,0,0,0,0,0,0,0
7001,16.200000,200246,59000,2,3,100,1,13,0,1,...,0,0,0,0,0,1,0,0,0,0
26422,14.833333,200944,178000,5,9,100,1,7,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20757,20.000000,202474,268000,0,7,640,10,15,0,0,...,0,0,0,0,0,0,0,0,1,0
32103,20.000000,201173,198000,2,10,1380,23,30,0,0,...,0,0,0,1,0,0,0,0,0,0
30403,20.000000,201177,148000,6,10,1260,21,13,1,0,...,0,0,0,1,0,0,0,0,0,0
21243,20.000000,200583,54900,4,7,1120,18,19,1,0,...,0,1,0,0,0,0,0,0,0,0


In [273]:
y_test

11681       0.046296
37084      19.028846
13307     117.995992
27110      26.849246
21600      56.310811
            ...     
37943       0.000000
33401      28.683789
35945    1319.061489
1647      251.102041
28512      27.586957
Name: 주문량, Length: 11490, dtype: float64

In [305]:
from keras.models import Sequential
from keras.layers import Dense

In [328]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=2269))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))



#model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
opt = tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(optimizer=opt, loss='mse', metrics=['mae'], )

model.summary()

ValueError: Could not interpret optimizer identifier: <tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop object at 0x000001FC91929048>

In [322]:
model.fit(X_train, y_train,
         epochs=100,
         batch_size=10,
         verbose=1)

Epoch 1/100
26810/26810 [==============================] - 8s 316us/step - loss: 6457675.2429 - mae: 340.6093
Epoch 2/100
26810/26810 [==============================] - 8s 313us/step - loss: 711091.1275 - mae: 302.1618
Epoch 3/100
26810/26810 [==============================] - 8s 313us/step - loss: 171637.6127 - mae: 295.4432
Epoch 4/100
26810/26810 [==============================] - 9s 321us/step - loss: 171816.7234 - mae: 295.3141
Epoch 5/100
26810/26810 [==============================] - 9s 342us/step - loss: 171717.3605 - mae: 295.1722
Epoch 6/100
26810/26810 [==============================] - 9s 318us/step - loss: 171760.1285 - mae: 295.2031
Epoch 7/100
26810/26810 [==============================] - 8s 309us/step - loss: 171770.2825 - mae: 294.9933
Epoch 8/100
26810/26810 [==============================] - 8s 303us/step - loss: 171779.7821 - mae: 295.4019
Epoch 9/100
26810/26810 [==============================] - 9s 345us/step - loss: 171718.0893 - mae: 295.3082
Epoch 10/100
26810

KeyboardInterrupt: 

# 그래프

In [97]:
실적데이터.drop('취급액', axis=1)

,노출,마더코드,상품코드,판매단가,요일,월,시간,시,일,분당평균시청률,...,건강기능,농수축,무형,생활용품,속옷,의류,이미용,잡화,주방,침구
0,20.0,100346,201072,39900,1,1,360,6,1,0.00,...,0,0,0,0,0,1,0,0,0,0
1,20.0,100346,201079,39900,1,1,360,6,1,0.00,...,0,0,0,0,0,1,0,0,0,0
2,20.0,100346,201072,39900,1,1,380,6,1,0.00,...,0,0,0,0,0,1,0,0,0,0
3,20.0,100346,201079,39900,1,1,380,6,1,0.00,...,0,0,0,0,0,1,0,0,0,0
4,20.0,100346,201072,39900,1,1,400,6,1,0.00,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38295,20.0,100448,201391,148000,1,12,1400,23,31,0.00,...,0,0,0,0,0,0,0,0,1,0
38296,20.0,100448,201383,178000,1,12,1420,23,31,2.06,...,0,0,0,0,0,0,0,0,1,0
38297,20.0,100448,201390,168000,1,12,1420,23,31,2.06,...,0,0,0,0,0,0,0,0,1,0
38298,20.0,100448,201384,158000,1,12,1420,23,31,2.06,...,0,0,0,0,0,0,0,0,1,0


In [96]:
실적데이터.columns

Index(['노출', '마더코드', '상품코드', '판매단가', '취급액', '요일', '월', '시간', '시', '일',
       '분당평균시청률', '주문량', '수목금토', '상반기', '하반기', '저가', '중가', '고가', '가구', '가전',
       '건강기능', '농수축', '무형', '생활용품', '속옷', '의류', '이미용', '잡화', '주방', '침구'],
      dtype='object')

In [ ]:
실적데이터.pivot_table(index=???, columns='???', values='???', aggfunc='???')

# 평가데이터 취급액 예측

In [83]:
평가데이터 = 평가데이터1.copy()
# 컬럼명 변경
평가데이터.columns = ['방송일시', '노출', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액']

# 결측치 제거
평가데이터.취급액 = 평가데이터.취급액.fillna(0)
평가데이터.노출 = 평가데이터.노출.fillna(method = 'ffill')


평가데이터['요일'] = 평가데이터['방송일시'].dt.dayofweek
평가데이터['월']  = 평가데이터['방송일시'].dt.month
평가데이터['시간']  = 평가데이터['방송일시'].dt.time
평가데이터['시']  = 평가데이터['방송일시'].dt.hour
평가데이터['일'] = 평가데이터['방송일시'].dt.day

평가데이터['방송시간'] = 평가데이터.방송일시.apply(lambda x : str(x)).apply(lambda x : x.split(' ')[1][:])
평가데이터['시간'] = 평가데이터.시간.apply(lambda x : str(x)).apply(lambda x : int(x.split(':')[0])*60 + int(x.split(':')[1]))
평가데이터['방송날짜'] = 평가데이터.방송일시.apply(lambda x : str(x)).apply(lambda x : x.split(' ')[0])


# 시간 데이터 컬럼 삭제
평가데이터.drop(['방송시간', '방송일시', '방송날짜',], axis=1, inplace=True)

# 상품명 컬럼 삭제
평가데이터.drop('상품명', axis=1, inplace=True)

# 요일 칼럼화
# 월 화 수 목 금 토 일
평가데이터 = pd.concat([평가데이터, pd.get_dummies(평가데이터.요일)], axis=1)
평가데이터.columns = ['노출', '마더코드', '상품코드', '상품군', '판매단가', '취급액', '요일', '월', '시간',
                 '시', '일', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
평가데이터.drop('요일', axis=1, inplace=True)

# 월 칼럼화
# 1 ~ 12 -> 월_1, 월_2, ~ ,월_12
#평가데이터 = pd.concat([평가데이터, pd.get_dummies(평가데이터.월, prefix='월')], axis=1)
#평가데이터.drop('월', axis=1, inplace=True)

# 일 칼럼화
# 1 ~ 31 -> 일_1, 일_2, ~, 일_31
#평가데이터 = pd.concat([평가데이터, pd.get_dummies(평가데이터.일, prefix='일')], axis=1)
#평가데이터.drop('일', axis=1, inplace=True)

# 시 칼럼화
# 1 ~ 23 -> 시_1, 시_2, ~, 시_23
#평가데이터 = pd.concat([평가데이터, pd.get_dummies(평가데이터.시, prefix='시')], axis=1)
#평가데이터.drop('시', axis=1, inplace=True)

# 상품군 컬럼화
# ['가구', '가전', '건강기능', '농수축', '무형', '생활용품', '속옷', '의류', '이미용', '잡화','주방', '침구']
평가데이터 = pd.concat([평가데이터, pd.get_dummies(평가데이터.상품군)], axis=1)
평가데이터.drop('상품군', axis=1, inplace=True)

평가데이터

,노출,마더코드,상품코드,판매단가,취급액,월,시간,시,일,월요일,...,건강기능,농수축,무형,생활용품,속옷,의류,이미용,잡화,주방,침구
0,20.000000,100650,201971,59800,0.0,6,380,6,1,1,...,0,0,0,0,0,1,0,0,0,0
1,20.000000,100650,201971,59800,0.0,6,400,6,1,1,...,0,0,0,0,0,1,0,0,0,0
2,20.000000,100650,201971,59800,0.0,6,420,7,1,1,...,0,0,0,0,0,1,0,0,0,0
3,20.000000,100445,202278,69900,0.0,6,440,7,1,1,...,0,0,0,0,1,0,0,0,0,0
4,20.000000,100445,202278,69900,0.0,6,460,7,1,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,20.000000,100660,201989,0,0.0,7,20,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2887,20.000000,100660,201989,0,0.0,7,40,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2888,20.000000,100660,201989,0,0.0,7,60,1,1,0,...,0,0,1,0,0,0,0,0,0,0
2889,20.000000,100261,200875,69900,0.0,7,80,1,1,0,...,0,0,0,0,0,1,0,0,0,0


In [84]:
X_test = 평가데이터.drop('취급액', axis=1)

In [22]:
X_train = 평가데이터.drop('')

2891

In [52]:
평가데이터

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN
